Create a results dataframe of a bowling tournament
==

In [2]:
# import libraries
import pandas as pd

# Input after tournament

In [3]:
city = 'Eindhoven'
date = "10-08-2018"

In [120]:
sheet_dict_string = [{'player':'Vlad', 'game':1, 'opponent':'Don', 'score_string':'9/7/639/9-9/639-9/8/9'},
                    {'player':'Dymytry', 'game':1, 'opponent':'Brunswick', 'score_string':'X 9-8/X 9-X X 8-526/5'},
                    {'player':'Kaspareski', 'game':1, 'opponent':None, 'score_string':'X X 43427-X 9/-/X 7- '},
                    {'player':'Don', 'game':1, 'opponent':'Vlad', 'score_string':'729/7-9/X X 81X X 9/9'},
                    {'player':'Brunswick', 'game':1, 'opponent':'Dymytry', 'score_string':'7-X 53-6317/549/8-13 '},
                    
                    {'player':'Vlad', 'game':2, 'opponent':'Brunswick', 'score_string':'811/639/8/X X 72519- '},
                    {'player':'Dymytry', 'game':2, 'opponent':'Kaspareski', 'score_string':'9/18X 9-3-339-338-7- '},
                    {'player':'Kaspareski', 'game':2, 'opponent':'Dymytry', 'score_string':'9/3/9/X 728-9/7272X1-'},
                    {'player':'Don', 'game':2, 'opponent':None, 'score_string':'9-629/2518X X 9-7181 '},
                    {'player':'Brunswick', 'game':2, 'opponent':'Vlad', 'score_string':'439-6-X 3/6331719--7 '},
                     
                    {'player':'Vlad', 'game':3, 'opponent':None, 'score_string':'X 7/71X 8-8/9/X X 6- '},
                    {'player':'Dymytry', 'game':3, 'opponent':'Don', 'score_string':'61X X 728/X 9/6/7/9/8'},
                    {'player':'Kaspareski', 'game':3, 'opponent':'Brunswick', 'score_string':'9-637-81718-9/8/7-34 '},
                    {'player':'Don', 'game':3, 'opponent':'Dymytry', 'score_string':'X 817-8-81278-9/8-34 '},
                    {'player':'Brunswick', 'game':3, 'opponent':'Kaspareski', 'score_string':'2-4-7--/6---9/818-72'},
                     
                    {'player':'Vlad', 'game':4, 'opponent':'Dymytry', 'score_string':'817/4/549-36X X X 71 '},
                    {'player':'Dymytry', 'game':4, 'opponent':'Vlad', 'score_string':'9/9/9/62X 6/16619/8/7'},
                    {'player':'Kaspareski', 'game':4, 'opponent':'Don', 'score_string':'717/9-9-X 128/44448- '},
                    {'player':'Don', 'game':4, 'opponent':'Kaspareski', 'score_string':'9/5361X 7281X 9/9/8/8'},
                    {'player':'Brunswick', 'game':4, 'opponent':None, 'score_string':'X 72X 71X 515-X 8/XX9'}
                    
                    ]

# Functions

In [109]:
def valid(game):
    """Check if a game string is valid.
    A valid game string is 21 chars.

    Keyword arguments:
    game -- string of game scores
    """
    if len(game) == 21:
        return True
    return False

def count_event(game, event):
    """Count the number of an event thrown in the game
    
    Keyword arguments:
    game -- a valid game string
    event -- strike
             spare
             gutter
             rotterdam
    
    Return:
    total events -- a int of the total event count
    """
    events = {}
    events['strike'] = 'X'
    events['spare'] = '/'
    events['gutter'] = '-'
    events['rotterdam'] = '-/'
    
    return game.count(events[event])

def score(game):
    """Calculate the scores per frame and in total.
    
    Foreach frame there are three possible scoring rules:
    
    
    Rule A. The score for a frame is the total pins bowled over during that frame, if the number is less than ten (an open frame, or error or split depending some other rules beyond the scope of this problem).
    Rule B. If all ten pins are bowled over on the first delivery (a strike), the score for that frame is 10 + the next two deliveries.
    Rule C. If all ten pins are bowled over between the first two deliveries (a spare), the score for that frame is 10 + the next delivery.

    
    Keyword arguments:
    game -- a valid game string
    
    Return:
    results -- a dict with a list of the frames and a list of 10 elements with score tuples
    """
    
    # List to return
    score = list()
    
    # Replace the dash for a zero
    game = game.replace('-','0')
    
    # Total score
    total = 0
    
    # Break the game string in frames
    n = 2
    frames = [game[i:i+n] for i in range(0, len(game)-3, n)]
    frames.append(game[-3:])
    
    # Loop through the frames to apply a score
    # TODO: KEY INDEX ENDING IS FRAME 9 & 10
    for key, frame in enumerate(frames):
        if key < 9:
            if frame[0] == 'X':
                # Rule B should be applied
                frame_score = 10
                
                if key is not 8:
                
                    next_deliveries = frames[key+1] + frames[key+2]

                    if next_deliveries[0] == 'X':
                        frame_score+= 10
                        if next_deliveries[2] == 'X':
                            frame_score+= 10
                        else:
                            frame_score+= int(next_deliveries[2])
                    else:
                        if next_deliveries[1] == '/':
                            frame_score+= 10
                        else:
                            frame_score+= int(next_deliveries[0]) + int(next_deliveries[1])
                else:
                    next_deliveries = frames[key+1]
                    
                    if next_deliveries[1] == '/':
                        frame_score+=10
                    else:
                        for n in next_deliveries[:2]:
                            if n == 'X':
                                frame_score+=10
                            else:
                                frame_score+=int(n)
            elif frame[1] == '/':
                # Rule C should be applied
                next_delivery = frames[key+1][0]
                next_delivery = next_delivery.replace('X','10')
                frame_score = 10 + int(next_delivery)
            else:
                # Rule A should be applied
                frame_score = int(frame[0]) + int(frame[1])
                
        else:
            last = list(frames[9])
            for k,v in enumerate(last):
                if v == ' ':
                    last[k] = 0
                if v == 'X':
                    last[k] = 10
                elif v == '/':
                    last[k] = 10 - int(last[k-1])
            last = [ int(x) for x in last ]
            frame_score = sum(last)
            
        total+= frame_score
        score.append((frame_score, total))
        
    # Undo the dash zero
    frames = [frame.replace('0','-') for frame in frames]
    return {'frames':frames,'score':score}


def who_won(row):
    """Who won the personal inside game
    
    Keyword arguments:
    row -- dataframe game row per player
    
    Return:
    True or False -- a boolean if the player had won
    """
    if row['opponent'] is not None:
        won = False
        if player_game_df.query("game == {} & player == '{}'".format(row['game'], row['opponent']))['result'].values[0] < row['result']:
            won = True
        return won

In [121]:
# calculates scores from the sheet_dict_string
scores = list()
player_game_result = list()

for i in sheet_dict_string:
    result = score(i['score_string'])
    
    player_game_dict = {}
    player_game_dict['player'] = i['player']
    player_game_dict['game'] = i['game']
    player_game_dict['opponent']= i['opponent']
    
    # Can only be calculated after all players results are build
    player_game_dict['won']= None
    player_game_dict['position']= None
    
    player_game_dict['hundred'] = None
    player_game_dict['strikes']= count_event(i['score_string'],'strike')
    player_game_dict['spares']= count_event(i['score_string'],'spare')
    player_game_dict['gutter']= count_event(i['score_string'],'gutter')
    player_game_dict['rotterdam']= count_event(i['score_string'],'rotterdam')
    player_game_dict['city'] = city
    player_game_dict['date'] = date
    player_game_dict['result'] = result['score'][9][1]
    
    for k,v in enumerate(result['frames']):
        my_dict = {}
        my_dict['player'] = i['player']
        my_dict['game'] = i['game']
        my_dict['city'] = city
        my_dict['date'] = date
        my_dict['frame'] = k+1
        my_dict['thrown'] = v
        my_dict['points'] = result['score'][k][0]
        my_dict['cumulative'] = result['score'][k][1]
        
        if result['score'][k][1] > 99 and player_game_dict['hundred'] is None:
            player_game_dict['hundred'] = my_dict['frame'] = k+1
        
        
        scores.append(my_dict)
        
    player_game_result.append(player_game_dict)

# generate dataframes
frames_df = pd.DataFrame(data=scores, columns=scores[0].keys()) 
player_game_df = pd.DataFrame(data=player_game_result, columns=player_game_result[0].keys())

# Was there a opponent? Who won?
player_game_df['won'] = player_game_df.apply(lambda x: who_won(x), axis=1)

# Position per player

# Beerframes

# Turkeys




# write the dataframes to .csv files for presentation in Power BI
frames_df.to_csv('result_data/{}_{}_FRAMES.csv'.format(city, date))

In [122]:
frames_df.shape

(200, 8)

In [123]:
player_game_df

,player,game,opponent,won,position,hundred,strikes,spares,gutter,rotterdam,city,date,result
0,Vlad,1,Don,False,None,8.0,0,6,2,0,Eindhoven,10-08-2018,141
1,Dymytry,1,Brunswick,True,None,6.0,4,2,3,0,Eindhoven,10-08-2018,152
2,Kaspareski,1,None,None,None,8.0,4,2,3,1,Eindhoven,10-08-2018,135
3,Don,1,Vlad,True,None,6.0,4,3,1,0,Eindhoven,10-08-2018,177
4,Brunswick,1,Dymytry,False,None,NaN,1,2,3,0,Eindhoven,10-08-2018,97
5,Vlad,2,Brunswick,True,None,7.0,2,3,1,0,Eindhoven,10-08-2018,142
6,Dymytry,2,Kaspareski,False,None,NaN,1,1,5,0,Eindhoven,10-08-2018,87
7,Kaspareski,2,Dymytry,True,None,7.0,2,4,2,0,Eindhoven,10-08-2018,134
8,Don,2,None,None,None,8.0,2,1,2,0,Eindhoven,10-08-2018,119
9,Brunswick,2,Vlad,False,None,NaN,1,1,4,0,Eindhoven,10-08-2018,95


In [128]:
player_game_df.groupby("player")['spares'].sum()

player
Brunswick      6
Don            9
Dymytry       14
Kaspareski    10
Vlad          14
Name: spares, dtype: int64